In [ ]:
#importing libraries

import numpy as np
from pandas import DataFrame
import pandas_datareader as web
import keras
import plotly.graph_objects as go
import tensorflow as tf
import pandas as pd
import itertools
from keras.models import Sequential
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.layers import Dense,LSTM
from __future__ import absolute_import,division,print_function
import os
import math

# Stock List - Add name of stocks for prediction

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
name_list = ['MARUTI.NS','NESTLEIND.NS','GAIL.NS','INDUSINDBK.NS','SBIN.NS','TITAN.NS','TCS.NS','BHARTIARTL.NS','KOTAKBANK.NS','ICICIBANK.NS','LT.NS','HINDALCO.NS','AMBUJACEM.NS','ONGC.NS','ULTRACEMCO.NS','HEROMOTOCO.NS','TECHM.NS','BAJFINANCE.NS','ZEEL.NS','HDFC.NS','IOC.NS','M&M.NS','RELIANCE.NS','YESBANK.NS','CIPLA.NS','GRASIM.NS','INFY.NS','NTPC.NS','BAJAJFINSV.NS','WIPRO.NS','POWERGRID.NS','ADANIPORTS.NS','ASIANPAINT.NS','AUROPHARMA.NS','AXISBANK.NS','BAJAJ-AUTO.NS','BPCL.NS','INFRATEL.NS','BOSCHLTD.NS','UPL.NS','DRREDDY.NS','EICHERMOT.NS','HCLTECH.NS','HDFCBANK.NS','LUPIN.NS','HINDPETRO.NS','HINDUNILVR.NS','ITC.NS','IBULHSGFIN.NS','SUNPHARMA.NS']

In [ ]:
# getting stock data from yahoo API
df_list =[]
for i in range(len(name_list)):
    my_dataset = web.DataReader(name_list[i], data_source = 'yahoo', start = '2005-01-01' , end = '2020-08-01')
    df_list.append(my_dataset)

In [ ]:
for i in range(50):
    print(len(df_list[i]))

In [ ]:
def Repeat(x): 
    _size = len(x) 
    repeated = [] 
    for i in range(_size): 
        k = i + 1
        for j in range(k, _size): 
            if x[i] == x[j] and x[i] not in repeated: 
                repeated.append(x[i]) 
    return repeated 
print (Repeat(name_list)) 

[]


In [ ]:
# function to get the number of rows to train the model
def data_len_func(data):
    dataset = data.values
    training_data_len = math.ceil(len(dataset)*.8)
    return training_data_len

In [ ]:
close_training_data_len = []
high_training_data_len = []
low_training_data_len = []
open_training_data_len = []

In [ ]:
# getting the length of close , open,high,low data for training
for i in df_list:
  close_training_data_len.append(data_len_func(i.filter(['Close'])))
  open_training_data_len.append(data_len_func(i.filter(['Open'])))
  high_training_data_len.append(data_len_func(i.filter(['High'])))
  low_training_data_len.append(data_len_func(i.filter(['Low']))) 

In [ ]:
# getting the scaled data of close,open,high,low 
data_list = list()
dataset_list = list()
close_scaled_data_list = list()
high_scaled_data_list = list()
open_scaled_data_list = list()
low_scaled_data_list = list()
for i in df_list:
  data = i.filter(['Close'])
  data_list.append(data)
  dataset = data.values
  dataset_list.append(dataset)
  close_scaler = MinMaxScaler(feature_range= (0,1))
  close_scaled_data = close_scaler.fit_transform(dataset)
  close_scaled_data_list.append(close_scaled_data)

  data = i.filter(['High'])
  data_list.append(data)
  dataset = data.values
  dataset_list.append(dataset)
  high_scaler = MinMaxScaler(feature_range= (0,1))
  high_scaled_data = high_scaler.fit_transform(dataset)
  high_scaled_data_list.append(high_scaled_data)

  data = i.filter(['Open'])
  data_list.append(data)
  dataset = data.values
  dataset_list.append(dataset)
  open_scaler = MinMaxScaler(feature_range= (0,1))
  open_scaled_data = open_scaler.fit_transform(dataset)
  open_scaled_data_list.append(open_scaled_data)

  data = i.filter(['Low'])
  data_list.append(data)
  dataset = data.values
  dataset_list.append(dataset)
  low_scaler = MinMaxScaler(feature_range= (0,1))
  low_scaled_data = low_scaler.fit_transform(dataset)
  low_scaled_data_list.append(low_scaled_data)


In [ ]:
#create the training dataset func
def create_train_dataset_fun(scaled_data,training_data_len):
    train_data = scaled_data[0:training_data_len,:]
    x_train = []
    y_train = []
    for i in range(60,len(train_data)):
        x_train.append(train_data[i-60:i,0])
        y_train.append(train_data[i,0])
    return x_train,y_train

In [ ]:
# get the traning dataset for close,open,high,low
close_x_train_list = list()
close_y_train_list = list()
open_x_train_list = list()
open_y_train_list = list()
high_x_train_list = list()
high_y_train_list = list()
low_x_train_list = list()
low_y_train_list = list()
for i in range(len(df_list)):
  close_x_train, close_y_train = create_train_dataset_fun(close_scaled_data_list[i],close_training_data_len[i])
  close_x_train_list.append(close_x_train)
  close_y_train_list.append(close_y_train)
  open_x_train, open_y_train = create_train_dataset_fun(open_scaled_data_list[i],open_training_data_len[i])
  open_x_train_list.append(open_x_train)
  open_y_train_list.append(open_y_train)
  high_x_train, high_y_train = create_train_dataset_fun(high_scaled_data_list[i],high_training_data_len[i])
  high_x_train_list.append(high_x_train)
  high_y_train_list.append(high_y_train)
  low_x_train, low_y_train = create_train_dataset_fun(low_scaled_data_list[i],low_training_data_len[i])
  low_x_train_list.append(low_x_train)
  low_y_train_list.append(low_y_train)

In [ ]:
# convert all the x_train and y_train into numpy arrays
for i in range(len(df_list)):
  close_x_train_list[i],close_y_train_list[i] = np.array(close_x_train_list[i]),np.array(close_y_train_list[i])
  open_x_train_list[i],open_y_train_list[i] = np.array(open_x_train_list[i]),np.array(open_y_train_list[i])
  high_x_train_list[i],high_y_train_list[i] = np.array(high_x_train_list[i]),np.array(high_y_train_list[i])
  low_x_train_list[i],low_y_train_list[i] = np.array(low_x_train_list[i]),np.array(low_y_train_list[i])


In [ ]:
#reshape the data in 3D
for i in range(len(df_list)):
  close_x_train_list[i] = np.reshape(close_x_train_list[i], (close_x_train_list[i].shape[0],close_x_train_list[i].shape[1],1))
  open_x_train_list[i] = np.reshape(open_x_train_list[i], ( open_x_train_list[i].shape[0],open_x_train_list[i].shape[1],1))
  high_x_train_list[i] = np.reshape(high_x_train_list[i], (high_x_train_list[i].shape[0],high_x_train_list[i].shape[1],1))
  low_x_train_list[i] = np.reshape(low_x_train_list[i], (low_x_train_list[i].shape[0],low_x_train_list[i].shape[1],1))
  close_x_train[i].shape

## Function for checking NAN, <1, >1 in x_train_list

In [ ]:
def check_list(train_list):
  count = 0
  count_1 = 0
  for r in range(train_list.shape[0]):
    for c in range(train_list.shape[1]):
      if (math.isnan(train_list[r][c])):
        train_list[r][c] = 0
        count = count+1
      elif train_list[r][c] == 1:
        train_list[r][c] = 0.99
        count_1 = count_1 +1
  print(count,count_1)

#if close_x_train_list[0][i] <= 0:
 # print("lol")

In [ ]:
# Implementing the function above
for i in range(len(df_list)):
  check_list(close_x_train_list[i])
  check_list(open_x_train_list[i])
  check_list(high_x_train_list[i])
  check_list(low_x_train_list[i])

0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 60
0 60
0 60
0 60
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 60
0 60
0 0
0 60
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 60
0 0
0 60
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 60
0 0
0 60
0 60
0 0
0 0
0 0
0 0
0 60
0 60
0 60
0 60
0 0
0 60
0 60
0 60
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 60
0 60
0 0
0 0
0 0
0 0
0 0
0 60
0 60
0 60
0 60
0 0
0 0
0 0
0 0
0 60
0 0
0 0
0 60
0 60
0 60
0 60
0 60
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 60
0 60
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 60
0 0
0 60
0 60
0 60
0 60
0 60
0 60


In [ ]:
def check_array(train_list):
    count = 0
    count_1 = 0
    for r in range(train_list.shape[0]):
      if (math.isnan(train_list[r])):
          #print('lol')
        train_list[r] = 0
        count = count+1
      elif train_list[r] == 1:
        train_list[r] = 0.99
        count_1 = count_1 +1
      elif train_list[r] < 0:
        train_list[r] = 0
    print(count,count_1)

In [ ]:
for i in range(len(df_list)):
  check_array(close_y_train_list[i])
  check_array(open_y_train_list[i])
  check_array(high_y_train_list[i])
  check_array(low_y_train_list[i])

0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 1
0 1
0 1
0 1
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 1
0 1
0 0
0 1
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 1
0 0
0 1
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 1
0 0
0 1
0 1
0 0
0 0
0 0
0 0
0 1
0 1
0 1
0 1
0 0
0 1
0 1
0 1
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 1
0 1
0 0
0 0
0 0
0 0
0 0
0 1
0 1
0 1
0 1
0 0
0 0
0 0
0 0
0 1
0 0
0 0
0 1
0 1
0 1
0 1
0 1
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 1
0 1
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 1
0 0
0 1
0 1
0 1
0 1
0 1
0 1


# Training the model and saving the weights into ckpt file


In [ ]:
name_list[17]

'BAJFINANCE.NS'

In [ ]:
for i in range(len(name_list)):
  print(name_list[i])
  close_model = Sequential()
  close_model.add(LSTM(50,return_sequences=True,input_shape = (close_x_train_list[i].shape[1],1)))
  close_model.add(LSTM(50,return_sequences= False))
  close_model.add(Dense(25))
  close_model.add(Dense(1))
  close_model.compile(optimizer = 'adam',loss = 'mean_squared_error')
  

  #build the model for high
  high_model = Sequential()
  high_model.add(LSTM(50,return_sequences=True,input_shape = (high_x_train_list[i].shape[1],1)))
  high_model.add(LSTM(50,return_sequences= False))
  high_model.add(Dense(25))
  high_model.add(Dense(1))
  high_model.compile(optimizer = 'adam',loss = 'mean_squared_error')
  
  
  
  #build the model for low
  low_model = Sequential()
  low_model.add(LSTM(50,return_sequences=True,input_shape = (low_x_train_list[i].shape[1],1)))
  low_model.add(LSTM(50,return_sequences= False))
  low_model.add(Dense(25))
  low_model.add(Dense(1))
  low_model.compile(optimizer = 'adam',loss = 'mean_squared_error')
  

  #build the model for open
  open_model = Sequential()
  open_model.add(LSTM(50,return_sequences=True,input_shape = (open_x_train_list[i].shape[1],1)))
  open_model.add(LSTM(50,return_sequences= False))
  open_model.add(Dense(25))
  open_model.add(Dense(1))
  open_model.compile(optimizer = 'adam',loss = 'mean_squared_error')
  

  
  train_list=[close_model,open_model,high_model,low_model]
  close_path = '/content/drive/My Drive/lusip/'+name_list[i] +'/close_weight/close.ckpt'
  open_path = '/content/drive/My Drive/lusip/'+name_list[i] +'/open_weight/open.ckpt'
  high_path = '/content/drive/My Drive/lusip/'+name_list[i] +'/high_weight/high.ckpt'
  low_path = '/content/drive/My Drive/lusip/'+name_list[i] +'/low_weight/low.ckpt'

  close_dir = os.path.dirname(close_path)
  cp_callback = tf.keras.callbacks.ModelCheckpoint(close_path,save_weights_only=True,verbose= 1)
  close_model.fit(close_x_train_list[i],close_y_train_list[i],batch_size=1,epochs=10,callbacks=[cp_callback])

  low_dir = os.path.dirname(low_path)
  cp_callback = tf.keras.callbacks.ModelCheckpoint(low_path,save_weights_only=True,verbose= 1)
  low_model.fit(low_x_train_list[i],low_y_train_list[i],batch_size=1,epochs=10,callbacks=[cp_callback])

  open_dir = os.path.dirname(open_path)
  cp_callback = tf.keras.callbacks.ModelCheckpoint(open_path,save_weights_only=True,verbose= 1)
  open_model.fit(open_x_train_list[i],open_y_train_list[i],batch_size=1,epochs=10,callbacks=[cp_callback])

  high_dir = os.path.dirname(high_path)
  cp_callback = tf.keras.callbacks.ModelCheckpoint(high_path,save_weights_only=True,verbose= 1)
  high_model.fit(high_x_train_list[i],high_y_train_list[i],batch_size=1,epochs=10,callbacks=[cp_callback])
  


ULTRACEMCO.NS
Epoch 1/10
3002/3007 [============================>.] - ETA: 0s - loss: 8.9396e-04
Epoch 00001: saving model to /content/drive/My Drive/lusip/ULTRACEMCO.NS/close_weight/close.ckpt
3007/3007 [==============================] - 18s 6ms/step - loss: 8.9352e-04
Epoch 2/10
3003/3007 [============================>.] - ETA: 0s - loss: 3.4952e-04
Epoch 00002: saving model to /content/drive/My Drive/lusip/ULTRACEMCO.NS/close_weight/close.ckpt
3007/3007 [==============================] - 18s 6ms/step - loss: 3.4916e-04
Epoch 3/10
3001/3007 [============================>.] - ETA: 0s - loss: 2.4541e-04
Epoch 00003: saving model to /content/drive/My Drive/lusip/ULTRACEMCO.NS/close_weight/close.ckpt
3007/3007 [==============================] - 17s 6ms/step - loss: 2.4501e-04
Epoch 4/10
3004/3007 [============================>.] - ETA: 0s - loss: 2.3349e-04
Epoch 00004: saving model to /content/drive/My Drive/lusip/ULTRACEMCO.NS/close_weight/close.ckpt
3007/3007 [========================

KeyboardInterrupt: ignored

In [ ]:
# create the test dataset func
def create_test_dataset_func(scaled_data,training_data_len,data):
    dataset = data.values
    #print(dataset)
    #print(scaled_data)
    test_data = scaled_data[training_data_len - 60: ,:]
    x_test = []
    y_test = dataset[training_data_len : , :]
    #print(len(test_data))
    for i in range(60,len(test_data)):
      x_test.append(test_data[i-60:i,0])
    #print()
    return x_test

In [ ]:
# get the test data
close_x_test_list = list()
high_x_test_list = list()
low_x_test_list = list()
open_x_test_list = list()
for i in range(len(df_list)):

  close_x_test = create_test_dataset_func(close_scaled_data_list[i],close_training_data_len[i],df_list[i].filter(['Close']))
  close_x_test_list.append(close_x_test)

  high_x_test = create_test_dataset_func(high_scaled_data_list[i],high_training_data_len[i],df_list[i].filter(['High']))
  high_x_test_list.append(high_x_test)

  low_x_test = create_test_dataset_func(low_scaled_data_list[i],low_training_data_len[i],df_list[i].filter(['Low']))
  low_x_test_list.append(low_x_test)

  open_x_test = create_test_dataset_func(open_scaled_data_list[i],open_training_data_len[i],df_list[i].filter(['Open']))
  open_x_test_list.append(open_x_test)



In [ ]:
#convert the data into numpy arrays
close_x_test_list_np = []
open_x_test_list_np = []
high_x_test_list_np = []
low_x_test_list_np = []

for i in range(len(df_list)):
  close_x_test = np.array(close_x_test_list[i])
  close_x_test_list_np.append(close_x_test)

  open_x_test = np.array(open_x_test_list[i])
  open_x_test_list_np.append(open_x_test) 

  high_x_test = np.array(high_x_test_list[i])
  high_x_test_list_np.append(high_x_test) 

  low_x_test = np.array(low_x_test_list[i])
  low_x_test_list_np.append(low_x_test) 

In [ ]:
#reshape the data
for i in range(len(df_list)):
  close_x_test_list_np[i] = np.reshape(close_x_test_list_np[i],
                                          (close_x_test_list_np[i].shape[0],
                                           close_x_test_list_np[i].shape[1],1))
  open_x_test_list_np[i] = np.reshape(open_x_test_list_np[i], (open_x_test_list_np[i].shape[0],open_x_test_list_np[i].shape[1],1))
  high_x_test_list_np[i] = np.reshape(high_x_test_list_np[i], (high_x_test_list_np[i].shape[0],high_x_test_list_np[i].shape[1],1))
  low_x_test_list_np[i] = np.reshape(low_x_test_list_np[i], (low_x_test_list_np[i].shape[0],low_x_test_list_np[i].shape[1],1))

In [ ]:
# get the model predicted values

close_predictions_list = []
high_predictions_list = []
open_predictions_list = []
low_predictions_list = []
print(10)
for i in range(len(df_list)):
    if(name_list[i] == 'TITAN.NS' or name_list[i] == 'ZEEL.NS'):
        continue
    print(name_list[i])
    close_path = '/content/drive/My Drive/lusip/'+ name_list[i] +'/close_weight/close.ckpt'
    open_path = '/content/drive/My Drive/lusip/'+ name_list[i] +'/open_weight/open.ckpt'
    high_path = '/content/drive/My Drive/lusip/'+ name_list[i] +'/high_weight/high.ckpt'
    low_path = '/content/drive/My Drive/lusip/'+ name_list[i] +'/low_weight/low.ckpt'
    print(close_path)
    close_model.load_weights(close_path)
    open_model.load_weights(open_path)
    low_model.load_weights(low_path)
    high_model.load_weights(high_path)
    
    close_predictions = close_model.predict(close_x_test_list_np[i]) 
    close_predictions = close_scaler.inverse_transform(close_predictions)
    close_predictions_list.append(close_predictions)

    high_predictions = high_model.predict(high_x_test_list_np[i]) 
    high_predictions = high_scaler.inverse_transform(high_predictions)
    high_predictions_list.append(high_predictions)

    open_predictions = open_model.predict(open_x_test_list_np[i]) 
    open_predictions = open_scaler.inverse_transform(open_predictions)
    open_predictions_list.append(open_predictions)

    low_predictions = low_model.predict(low_x_test_list_np[i]) 
    low_predictions = low_scaler.inverse_transform(low_predictions)

    low_predictions_list.append(low_predictions)

10
MARUTI.NS
/content/drive/My Drive/lusip/MARUTI.NS/close_weight/close.ckpt
NESTLEIND.NS
/content/drive/My Drive/lusip/NESTLEIND.NS/close_weight/close.ckpt
GAIL.NS
/content/drive/My Drive/lusip/GAIL.NS/close_weight/close.ckpt
INDUSINDBK.NS
/content/drive/My Drive/lusip/INDUSINDBK.NS/close_weight/close.ckpt
SBIN.NS
/content/drive/My Drive/lusip/SBIN.NS/close_weight/close.ckpt
TCS.NS
/content/drive/My Drive/lusip/TCS.NS/close_weight/close.ckpt
BHARTIARTL.NS
/content/drive/My Drive/lusip/BHARTIARTL.NS/close_weight/close.ckpt
KOTAKBANK.NS
/content/drive/My Drive/lusip/KOTAKBANK.NS/close_weight/close.ckpt
ICICIBANK.NS
/content/drive/My Drive/lusip/ICICIBANK.NS/close_weight/close.ckpt


In [ ]:
#predict the values for future
count =0
predicted_df = []
k =0
for i in df_list:
  close_train = i.filter(['Close'])[:close_training_data_len[k]] 
  close_valid = i.filter(['Close'])[close_training_data_len[k]:]
  close_valid['Predictions'] = pd.Series(close_predictions_list)

  high_train = i.filter(['High'])[:high_training_data_len[k]] 
  high_valid = i.filter(['High'])[high_training_data_len[k]:]
  high_valid['Predictions'] = pd.Series(high_predictions_list)

  low_train = i.filter(['Low'])[:low_training_data_len[k]] 
  low_valid = i.filter(['Low'])[low_training_data_len[k]:]
  low_valid['Predictions'] = pd.Series(low_predictions_list)

  open_train = i.filter(['Open'])[:open_training_data_len[k]] 
  open_valid = i.filter(['Open'])[open_training_data_len[k]:]
  open_valid['Predictions'] = pd.Series(open_predictions_list) 
  

  close_predictions = close_predictions.flatten()
  low_predictions = low_predictions.flatten()
  high_predictions = high_predictions.flatten()
  open_predictions = open_predictions.flatten()
  list_of_tuples = list(zip(close_predictions,low_predictions,high_predictions,open_predictions )) 
  df = pd.DataFrame(list_of_tuples, columns = ['Close', 'Low','High','Open'])
  predicted_df.append(df)
  count+=1
  print(name_list[k])
  name = str(name_list[k]) + ".csv"
  df.to_csv(name) 
  k+=1
    

MARUTI.NS
NESTLEIND.NS
GAIL.NS
INDUSINDBK.NS
SBIN.NS
TITAN.NS
TCS.NS
BHARTIARTL.NS
KOTAKBANK.NS
ICICIBANK.NS


name_list

In [ ]:
# print output function 
def print_output(var1, var2, var3, var4, tf,tf_df):
  #list1 = [("Open", var1), ("High", var2), ("Low", var3), ("Close", var4)]
  list1=[var1[0][0],var2[0][0],var3[0][0],var4[0][0]]
  df = pd.DataFrame(data = pd.Series(list1))
  df = df.T

  df=df.rename(columns = {0:'Open', 1:'High',2:'Low', 3:'Close'})
  df['truthful_beh_ori'] = pd.Series([tf_df]) 
  df['T_Behave'] = pd.Series([tf])
  count=0
  if(tf == tf_df):
    print("MATCHED")
  pd.set_option('display.max_columns', None)
  print(df)
  
  return df
  

In [ ]:
# function to get the updown behaviour of the prediction
def up_down(j,var):
  list2=[]
  behaviour_list = []
  result = " "
  length_ = (j['Close'].size) - 1
  #print(length_)
  behaviour=(var - j['Close'][length_]) / (j['Close'][length_])
  if behaviour>=0.005:
    result="UP"
  elif behaviour<= -0.005:
    result="Down"
  else:
    result="NA"

  return result

In [ ]:
# function to get the updown behaviour of the original dataset
def up_down_ori(j):
  list2=[]
  behaviour_list = []
  result = " "
  length_ = (j['Close'].size) - 1
  length_1 = (j['Close'].size)
  #print(length_)
  behaviour=(j['Close'][length_1-1] - j['Close'][length_-1]) / (j['Close'][length_-1])
  if behaviour>=0.005:
    result="UP"
  elif behaviour<= -0.005:
    result="Down"
  else:
    result="NA"

  return result

# Predicting the values for Next day 

In [ ]:
count =0
pO = []
pL=[]
pH=[]
pC =[]
ptf = []
ptf_df = []
for i in range(len(df_list)):
  pred_price_list_close = []
  pred_price_list_open = []
  pred_price_list_low = []
  pred_price_list_high = []
  #Get teh last 60 day closing price 
  last_60_days_c = df_list[i]['Close'][-60:].values
  last_60_days_o = df_list[i]['Open'][-60:].values
  last_60_days_l = df_list[i]['Low'][-60:].values
  last_60_days_h = df_list[i]['High'][-60:].values

  #Scale the data to be values between 0 and 1
  last_60_days_scaled_c = close_scaler.fit_transform(last_60_days_c.reshape(-1,1))
  last_60_days_scaled_o = open_scaler.fit_transform(last_60_days_o.reshape(-1,1))
  last_60_days_scaled_l = low_scaler.fit_transform(last_60_days_l.reshape(-1,1))
  last_60_days_scaled_h = high_scaler.fit_transform(last_60_days_h.reshape(-1,1))

  #Create an empty list
  X_test_c = []
  X_test_o = []
  X_test_l = []
  X_test_h = []

  #Append teh past 60 days
  X_test_c.append(last_60_days_scaled_c)
  X_test_o.append(last_60_days_scaled_o)
  X_test_l.append(last_60_days_scaled_l)
  X_test_h.append(last_60_days_scaled_h)

  #Convert the X_test data set to a numpy array
  X_test_c = np.array(X_test_c)
  X_test_o = np.array(X_test_o)
  X_test_l = np.array(X_test_l)
  X_test_h = np.array(X_test_h)

  #Reshape the data
  X_test_c = np.reshape(X_test_c, (X_test_c.shape[0], X_test_c.shape[1], 1))
  X_test_o = np.reshape(X_test_o, (X_test_o.shape[0], X_test_o.shape[1], 1))
  X_test_l = np.reshape(X_test_l, (X_test_l.shape[0], X_test_l.shape[1], 1))
  X_test_h = np.reshape(X_test_h, (X_test_h.shape[0], X_test_h.shape[1], 1))

  #Get the predicted scaled price
  pred_price_c = close_model.predict(X_test_c)
  pred_price_o = open_model.predict(X_test_o)
  pred_price_l = low_model.predict(X_test_l)
  pred_price_h = open_model.predict(X_test_h)
  print(name_list[i])
  #undo the scaling 
  pred_price_c = close_scaler.inverse_transform(pred_price_c)
  pred_price_list_close.append(pred_price_c)
  #print('close'  , pred_price_list_close)
  #print(pred_price_c)
  
  pred_price_o = open_scaler.inverse_transform(pred_price_o)
  pred_price_list_open.append(pred_price_o)
  #print('open' , pred_price_list_open)

  pred_price_l = low_scaler.inverse_transform(pred_price_l)
  pred_price_list_low.append(pred_price_l)
  #print('low' , pred_price_list_low)
 
  pred_price_h = high_scaler.inverse_transform(pred_price_h)
  pred_price_list_high.append(pred_price_h)
  #print('high' , pred_price_list_high)

  #print()
  tf_df =up_down_ori(df_list[i])
  tf = up_down(df_list[i],pred_price_c)
  #print()
  
  if(tf_df == tf):
    count+=1
  pO.append(pred_price_o)
  pH.append(pred_price_h)
  pL.append(pred_price_l)
  pC.append(pred_price_c)
  ptf.append(tf)
  ptf_df.append(tf_df)
  print_output(pred_price_o, pred_price_h, pred_price_l, pred_price_c, tf,tf_df)

  print()
  #print(type(pred_price_o[0]))

0
MARUTI.NS
          Open         High          Low        Close truthful_beh_ori  \
0  6152.905273  6261.604492  6069.656738  6094.686523               NA   

  T_Behave  
0     Down  

1
NESTLEIND.NS
MATCHED
           Open          High           Low         Close truthful_beh_ori  \
0  16560.896484  16701.845703  16400.277344  16564.101562               NA   

  T_Behave  
0       NA  

2
GAIL.NS
MATCHED
        Open       High       Low      Close truthful_beh_ori T_Behave
0  96.567917  98.175316  95.52977  96.269508               NA       NA

3
INDUSINDBK.NS
         Open        High         Low       Close truthful_beh_ori T_Behave
0  515.472778  521.072815  503.042664  509.569946               UP     Down

4
SBIN.NS
         Open        High        Low       Close truthful_beh_ori T_Behave
0  186.712692  193.101456  184.42366  188.777771               UP     Down

5
TITAN.NS
          Open        High          Low        Close truthful_beh_ori T_Behave
0  1031.977295  1039.666

In [ ]:
pO_ori = []
pC_ori = []
pL_ori = []
pH_ori = []
for i in range(48):
    pO_ori.append(df_list2[i]['Close'][len(df_list2[i]['Close'])-1])
    pC_ori.append(df_list2[i]['Close'][len(df_list2[i]['Close'])-1])
    pL_ori.append(df_list2[i]['Close'][len(df_list2[i]['Close'])-1])
    pH_ori.append(df_list2[i]['Close'][len(df_list2[i]['Close'])-1])

NameError: ignored

#Storing the file in the csv file

In [ ]:
list_of_tuples = list(zip(pO,pO_ori,pL,pL_ori,pH,pH_ori,pC,pC_ori,ptf,ptf_df,name_list[0:48]))
ans = pd.DataFrame(list_of_tuples, columns = ['predicted_open','original_Open' ,'predicted_Low','original_Low' ,'predicted_High','original_HIgh' ,'predicted_Close','original_Close' ,'predicted_behav','original_behav','company'])  
ans.to_csv('behav.csv')
print(ans)